### General Optimisation of Input Tokens 

In [170]:
from inseq import load_model
import torch
import torch.nn.functional as F
# Load model with gradient-based attribution
model = load_model(model="gpt2", attribution_method="saliency", device="cuda" if torch.cuda.is_available() else "cpu")
embedding_layer = model.model.get_input_embeddings()


In [171]:


def optimize_toward_target(input_text, target_token, step) : 
    """Change the input tokens in such way that the probability of getting the target_token is increased.
    The optimization is done for all input tokens at once (in parallel)

    Args:
        input_text (_type_): sequence of input text (str)
        target_token (_type_): the next token that amy be given by the model to compete the input text (str)
        step (_type_): factor by which the gradient is multiplied to move in the embedding space (int)

    Returns:
        _type_: new sequence of input text (str)
    """

    # Convert input text to token 
    inputs = model.tokenizer(input_text, return_tensors="pt")
    target_token = model.tokenizer.tokenize(" " + target_token.strip()) # make sur the target output word starts with a space to ahve a recognised token
    if len(target_token) == 1: 
        target_token = target_token[0]
    else: 
        print(f"Several tokens have been assigned to the target output word: {target_token}.")
        return None
    target_id = model.tokenizer.convert_tokens_to_ids(target_token) # convert the target token to ID

    # # For optimizing toward a phrase:
    # target_ids = [model.tokenizer.convert_tokens_to_ids(t) for t in ["lazy", "dog"]]
    # target_logits = sum(outputs.logits[0, -len(target_ids):, target_ids].diag())

    # Stop if target token is not part of the vocabulary of gpt2
    if target_id == model.tokenizer.unk_token_id:
        print(f"Warning: Target token '{target_token}' not found in vocabulary. Using UNK ID ({target_id}). Results might not be meaningful.")
        return None

    # Perform forward pass with gradient tracking
    with torch.set_grad_enabled(True):
        # Get embeddings and mark as requiring gradients (so that we retain the gradient for non-leaf tensors)
        embeddings = embedding_layer(inputs.input_ids)
        embeddings.retain_grad()
        embeddings.requires_grad_(True)
        
        # Forward pass through model
        outputs = model.model(
            inputs_embeds=embeddings,
            attention_mask=inputs.attention_mask
        )
        
        # Get logits for the target position (next token prediction)
        target_position = -1  # Position after last input token
        target_logit = outputs.logits[0, target_position, target_id]
        
        # Compute gradients d(logit)/d(embeddings)
        target_logit.backward()
        gradients = embeddings.grad.clone()

    # Modify embeddings with gradients
    # print("Comuted Gradients: ", gradients)
    for i in range(len(gradients[0])) : 
        gradients[0][i] = gradients[0][i] / (sum(abs(i) for i in gradients[0][i])) # compute 1 step gradient (divide by norm the gradient)

    modified_embeddings = embeddings + gradients*step

    # Nearest Neighbor Search
    all_embeddings = embedding_layer.weight #Vocab Embeddings; all tokens in vocabulary
    new_token_ids = []
    for i, emb in enumerate(modified_embeddings[0]) : 
        # find closest token in voc. Best to use cos similarity as it will ignore the magnitude of the embeddings and focus on the direction
        # cos(a, b) = (a . b) / (||a|| * ||b||)
        distances = F.cosine_similarity(emb, all_embeddings, dim=-1)

        # # Euclidean distance 
        # distances = -torch.norm(all_embeddings - emb, dim=1)
        # # Dot product (ignore magnitude normalization here)
        # distances = torch.matmul(all_embeddings, emb)

        new_id = torch.argmax(distances).item()
        new_token_ids.append(new_id)

    #Decode the new tokens
    new_tokens = model.tokenizer.convert_ids_to_tokens(new_token_ids)
    new_text = model.tokenizer.convert_tokens_to_string(new_tokens)
    print(f"Original text: {input_text}")
    print(f"Modified text: {new_text}")


    # Compare Probabilities
    with torch.no_grad():
        modified_outputs = model.model(
            inputs_embeds=modified_embeddings,
            attention_mask=inputs.attention_mask
        )
        original_probs = torch.softmax(outputs.logits[0, target_position], dim=-1)
        modified_probs = torch.softmax(modified_outputs.logits[0, target_position], dim=-1)

    print(f"Original probability of '{target_token}': {original_probs[target_id].item():.10f}")
    print(f"Modified probability of '{target_token}': {modified_probs[target_id].item():.10f}")

    return new_text


### Specific Input Token Optimisation

In [ ]:
from inseq import load_model
import torch
import torch.nn.functional as F
# Load model with gradient-based attribution
model = load_model(model="gpt2", attribution_method="saliency", device="cuda" if torch.cuda.is_available() else "cpu")
embedding_layer = model.model.get_input_embeddings()


In [ ]:

def optimize_single_input_token_toward_target(input_text, target_token, step, n_iteration, token_input_position) : 

    target_token = model.tokenizer.tokenize(" " + target_token.strip()) # make sur the target output word starts with a space to ahve a recognised token
    if len(target_token) == 1: 
        target_token = target_token[0]
    else: 
        print(f"Several tokens have been assigned to the target output word: {target_token}.")
        return None
    target_id = model.tokenizer.convert_tokens_to_ids(target_token) # convert the target token to ID

    if target_id == model.tokenizer.unk_token_id:
        print(f"Warning: Target token '{target_token}' not found in vocabulary. Using UNK ID ({target_id}). Results might not be meaningful.")
        return None
    
    original_text = input_text
    for i in range(n_iteration) :
        
        inputs = model.tokenizer(original_text, return_tensors="pt")

        with torch.set_grad_enabled(True):
            # Get embeddings and mark as requiring gradients (so that we retain the gradient for non-leaf tensors)
            embeddings = embedding_layer(inputs.input_ids)
            embeddings.retain_grad()
            embeddings.requires_grad_(True)
            
            # Forward pass through model
            outputs = model.model(
                inputs_embeds=embeddings,
                attention_mask=inputs.attention_mask
            )
            # Get logits for the target position (next token prediction)
            target_position = -1  # Position after last input token
            target_logit = outputs.logits[0, target_position, target_id]
            
            # Compute gradients d(logit)/d(embeddings)
            target_logit.backward()
            gradients = embeddings.grad.clone()

        # Modify embeddings with gradients
        # print("Comuted Gradients: ", gradients)
        for i in range(len(gradients[0])) : 
            gradients[0][i] = gradients[0][i] / (sum(abs(i) for i in gradients[0][i])) # compute 1 step gradient (divide by norm the gradient)

        modified_embeddings = embeddings

        modified_embeddings[0][token_input_position] += gradients[0][token_input_position]*step # only for specific input token
        # + 0.9*(gradients[0].mean(dim=0)) #consider the context (0.5*mean of the input embeddings)

        # Nearest Neighbor Search
        all_embeddings = embedding_layer.weight #Vocab Embeddings; all tokens in vocabulary
        new_token_ids = []
        distances = F.cosine_similarity(modified_embeddings[0][token_input_position], all_embeddings, dim=-1)
        new_id = torch.argmax(distances).item()
        new_token_ids.append(new_id)
        inputs['input_ids'][0][token_input_position] = new_token_ids[0]
        new_token_ids = inputs['input_ids']

        # print("DEBUG: ", new_token_ids)


        #Decode the new tokens
        new_tokens = model.tokenizer.convert_ids_to_tokens(new_token_ids[0])
        new_text = model.tokenizer.convert_tokens_to_string(new_tokens)
        print(f"Original text: {original_text}")
        print(f"Modified text: {new_text}")


        # Compare Probabilities
        with torch.no_grad():
            modified_outputs = model.model(
                inputs_embeds=modified_embeddings,
                attention_mask=inputs.attention_mask
            )
            original_probs = torch.softmax(outputs.logits[0, target_position], dim=-1)
            modified_probs = torch.softmax(modified_outputs.logits[0, target_position], dim=-1)

        print(f"Original probability of '{target_token}': {original_probs[target_id].item():.10f}")
        print(f"Modified probability of '{target_token}': {modified_probs[target_id].item():.10f}")

        original_text = new_text

    return new_text


In [159]:
input_text = "that black woman is"
target_token = "pretty"
optimize_single_input_token_toward_target(input_text=input_text, target_token=target_token, step=512, token_input_position=1, n_iteration=5)

Original text: that black woman is
Modified text: that red woman is
Original probability of 'Ġpretty': 0.0013306340
Modified probability of 'Ġpretty': 0.0020418002
Original text: that red woman is
Modified text: that thus woman is
Original probability of 'Ġpretty': 0.0031131522
Modified probability of 'Ġpretty': 0.0003907612
Original text: that thus woman is
Modified text: that thus woman is
Original probability of 'Ġpretty': 0.0001825135
Modified probability of 'Ġpretty': 0.0016098868
Original text: that thus woman is
Modified text: that thus woman is
Original probability of 'Ġpretty': 0.0001825135
Modified probability of 'Ġpretty': 0.0016098868
Original text: that thus woman is
Modified text: that thus woman is
Original probability of 'Ġpretty': 0.0001825135
Modified probability of 'Ġpretty': 0.0016098868


'that thus woman is'

### Integrated Gradients Method


In [173]:
from inseq import load_model
import torch
import torch.nn.functional as F
# Load model with gradient-based attribution
model = load_model(model="gpt2", attribution_method="integrated_gradients", device="cuda" if torch.cuda.is_available() else "cpu")
embedding_layer = model.model.get_input_embeddings()


In [213]:
def optimize_specific_input_token_with_IG_towards_target_1(input_text, target_token, step, n_iteration, input_token_pos) : 

    target_token = model.tokenizer.tokenize(" " + target_token.strip()) # make sur the target output word starts with a space to ahve a recognised token
    if len(target_token) == 1: 
        target_token = target_token[0]
    else: 
        print(f"Several tokens have been assigned to the target output word: {target_token}.")
        return None
    target_id = model.tokenizer.convert_tokens_to_ids(target_token) # convert the target token to ID

    if target_id == model.tokenizer.unk_token_id:
        print(f"Warning: Target token '{target_token}' not found in vocabulary. Using UNK ID ({target_id}). Results might not be meaningful.")
        return None
    
    original_text = input_text
    for i in range(n_iteration) :
        
        inputs = model.tokenizer(original_text, return_tensors="pt")

        with torch.set_grad_enabled(True):
            # Get embeddings and mark as requiring gradients (so that we retain the gradient for non-leaf tensors)
            embeddings = embedding_layer(inputs.input_ids)
            embeddings.retain_grad()
            embeddings.requires_grad_(True)
            
            # Forward pass through model
            outputs = model.model(
                inputs_embeds=embeddings,
                attention_mask=inputs.attention_mask
            )
            # Get logits for the target position (next token prediction)
            target_position = -1  # Position after last input token
            target_logit = outputs.logits[0, target_position, target_id]
            
            # Compute gradients d(logit)/d(embeddings)
            target_logit.backward()
            gradients = embeddings.grad.clone()

        # Modify embeddings with gradients
        # print("Comuted Gradients: ", gradients)
        for i in range(len(gradients[0])) : 
            gradients[0][i] = gradients[0][i] / (sum(abs(i) for i in gradients[0][i])) # compute 1 step gradient (divide by norm the gradient)

        modified_embeddings = embeddings

        modified_embeddings[0][input_token_pos] += gradients[0][input_token_pos]*step # only for specific input token
        # + 0.9*(gradients[0].mean(dim=0)) #consider the context (0.5*mean of the input embeddings)

        # Nearest Neighbor Search
        all_embeddings = embedding_layer.weight #Vocab Embeddings; all tokens in vocabulary
        new_token_ids = []
        distances = F.cosine_similarity(modified_embeddings[0][input_token_pos], all_embeddings, dim=-1)
        new_id = torch.argmax(distances).item()
        new_token_ids.append(new_id)
        inputs['input_ids'][0][input_token_pos] = new_token_ids[0]
        new_token_ids = inputs['input_ids']

        # print("DEBUG: ", new_token_ids)


        #Decode the new tokens
        new_tokens = model.tokenizer.convert_ids_to_tokens(new_token_ids[0])
        new_text = model.tokenizer.convert_tokens_to_string(new_tokens)
        print(f"Original text: {original_text}")
        print(f"Modified text: {new_text}")


        # Compare Probabilities
        with torch.no_grad():
            modified_outputs = model.model(
                inputs_embeds=modified_embeddings,
                attention_mask=inputs.attention_mask
            )
            original_probs = torch.softmax(outputs.logits[0, target_position], dim=-1)
            modified_probs = torch.softmax(modified_outputs.logits[0, target_position], dim=-1)

        print(f"Original probability of '{target_token}': {original_probs[target_id].item():.10f}")
        print(f"Modified probability of '{target_token}': {modified_probs[target_id].item():.10f}")

        original_text = new_text

    return new_text

In [ ]:
def optimize_specific_input_token_with_IG_towards_target_2(input_text, target_token, input_token_pos, n_iteration, step): 
    
    target_token = " " + target_token.strip()
    target_token =  model.tokenizer.tokenize(target_token)[0]
    target_id = model.tokenizer.convert_tokens_to_ids(target_token[0])

    current_text = input_text
    for i in range(n_iteration) :  
        original_text = current_text
        output = model.attribute(
            current_text, 
            f"{current_text}{target_token[0]}", 
            attributed_fn="probability", 
            step_scores=["probability"],
            show_progress=False
        )


        grad =  output.sequence_attributions[0].target_attributions[input_token_pos]
        grad = grad /  (torch.norm(grad, p=1))

        inputs = model.tokenizer(current_text, return_tensors='pt')
        embeddings = model.model.get_input_embeddings()(inputs.input_ids)
        modified_embeddings = embeddings[0, input_token_pos] + step*grad

        all_embeddings = model.model.get_input_embeddings().weight
        
        distances  = F.cosine_similarity(modified_embeddings[0, input_token_pos], 
                                         all_embeddings,  
                                         dim=-1)
        new_id = torch.argmax(distances).item()
        new_token = model.tokenizer.convert_ids_to_tokens([new_id])[0]

        current_tokens = model.tokenizer.tokenize(current_text)
        print("INPUT TOKENS: ", current_tokens)

        if (input_token_pos != 0) & (new_token[0] != "Ġ"): 
            new_token = "Ġ" + new_token

        current_tokens[input_token_pos] = new_token
        current_text = model.tokenizer.convert_tokens_to_string(current_tokens)
        print("DEBUG: ", new_token)


        orig_prob = output.sequence_attributions[0].step_scores['probability'][0].item()
        new_output = model.attribute(
            current_text,
            f"{current_text}{target_token[0]}",
            attributed_fn="probability",
            step_scores=["probability"],
            show_progress=False, 
        )
        new_prob = new_output.sequence_attributions[0].step_scores["probability"][0].item()
        
        print(f"Modified: {model.tokenizer.tokenize(original_text)[input_token_pos]} → {new_token}")
        print(f"Full text: {current_text}")
        print(f"P({target_token}): {orig_prob:.10f} → {new_prob:.10f} ({'↑' if new_prob > orig_prob else '↓'})")

    return current_text

        


In [242]:
input_text = "that black woman is"
target_token = "pretty" 

optimize_specific_input_token_with_IG_towards_target_2(input_text=input_text, target_token=target_token, input_token_pos=2, n_iteration=5, step=1)

INPUT TOKENS:  ['that', 'Ġblack', 'Ġwoman', 'Ġis']
DEBUG:  Ġ://
Modified: Ġwoman → Ġ://
Full text: that black :// is
P(Ġpretty): 0.0000000184 → 0.0000009801 (↑)
INPUT TOKENS:  ['that', 'Ġblack', 'Ġ:', '//', 'Ġis']
DEBUG:  Ġelsius
Modified: Ġ: → Ġelsius
Full text: that black elsius// is
P(Ġpretty): 0.0000009801 → 0.0000007678 (↓)
INPUT TOKENS:  ['that', 'Ġblack', 'Ġel', 's', 'ius', '//', 'Ġis']
DEBUG:  Ġ://
Modified: Ġel → Ġ://
Full text: that black ://sius// is
P(Ġpretty): 0.0000007678 → 0.0000007765 (↑)
INPUT TOKENS:  ['that', 'Ġblack', 'Ġ:', '//', 's', 'ius', '//', 'Ġis']
DEBUG:  Ġelsius
Modified: Ġ: → Ġelsius
Full text: that black elsius//sius// is
P(Ġpretty): 0.0000007765 → 0.0000011018 (↑)
INPUT TOKENS:  ['that', 'Ġblack', 'Ġel', 's', 'ius', '//', 's', 'ius', '//', 'Ġis']
DEBUG:  Ġ://
Modified: Ġel → Ġ://
Full text: that black ://sius//sius// is
P(Ġpretty): 0.0000011018 → 0.0000013532 (↑)


'that black ://sius//sius// is'

In [ ]:
torch.linalg.norm(gradients[0][0], ord=1)
sum(abs(i) for i in gradients[0][0])
gradients[0][0].abs().sum()

tensor(277.1607)

In [60]:

n = 20
iter_ = 0

input_text = "that black woman is"
target_token = "pretty"  # Token we want to make more favorable

new_input = input_text

while iter_ <= n : 
    new_input = optimize_toward_target(input_text=new_input, target_token=target_token, step = 5)
    if new_input == None :
        break

    iter_+=1



Original text: that black woman is
Modified text: that black woman is
Original probability of 'Ġpretty': 0.0013306340
Modified probability of 'Ġpretty': 0.0015035227
Original text: that black woman is
Modified text: that black woman is
Original probability of 'Ġpretty': 0.0013306340
Modified probability of 'Ġpretty': 0.0015035227
Original text: that black woman is
Modified text: that black woman is
Original probability of 'Ġpretty': 0.0013306340
Modified probability of 'Ġpretty': 0.0015035227
Original text: that black woman is
Modified text: that black woman is
Original probability of 'Ġpretty': 0.0013306340
Modified probability of 'Ġpretty': 0.0015035227
Original text: that black woman is
Modified text: that black woman is
Original probability of 'Ġpretty': 0.0013306340
Modified probability of 'Ġpretty': 0.0015035227
Original text: that black woman is
Modified text: that black woman is
Original probability of 'Ġpretty': 0.0013306340
Modified probability of 'Ġpretty': 0.0015035227
Orig

In [29]:
gradients[0][0] / sum(abs(i) for i in gradients[0][0])

tensor([-3.4591e-03, -2.0714e-04,  7.3177e-04, -1.0140e-03,  8.6816e-04,
        -2.0273e-03, -3.3901e-04,  1.4948e-03, -2.7162e-03, -2.9139e-03,
        -2.6725e-03,  5.4865e-04,  1.8390e-03,  1.7984e-03, -9.2928e-04,
         2.0346e-03,  3.7214e-03, -1.2461e-03, -2.0981e-04,  1.0690e-04,
         2.6508e-03, -3.0332e-03,  4.2063e-03, -1.6150e-03, -3.9101e-03,
        -3.3079e-04,  3.1795e-04, -1.1389e-03, -1.6902e-03,  4.0789e-03,
        -1.9345e-03, -4.4975e-04,  1.9578e-03,  7.3786e-04,  2.8309e-03,
         2.8711e-04, -4.3191e-04, -1.1351e-03,  4.2762e-03, -1.5933e-03,
         2.3057e-03,  2.2046e-03, -1.5345e-03,  1.3337e-03,  1.6590e-03,
        -6.4271e-03,  1.2239e-03, -1.7001e-03,  1.5255e-03, -1.8939e-04,
         3.0685e-04,  2.6848e-04, -1.3084e-04,  1.2047e-03, -1.5419e-03,
        -1.9634e-04,  1.1819e-03, -1.9331e-03, -1.6991e-03,  4.2700e-03,
        -1.0560e-03,  3.0874e-03,  1.4234e-03,  1.8472e-04,  1.5561e-03,
         7.2009e-04,  4.2875e-03,  1.2394e-03, -2.7

In [85]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# 1. Load model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# 2. Input text
input_text = "The quick brown fox jumps over the"

# 3. Tokenize and get model prediction
inputs = tokenizer(input_text, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
    predicted_token_id = torch.argmax(outputs.logits[0, -1]).item()  # Get last token prediction

# 4. Decode the predicted token
predicted_token = tokenizer.decode([predicted_token_id])
print(f"Input: '{input_text}'")
print(f"Model's predicted next token: '{predicted_token}'")

Input: 'The quick brown fox jumps over the'
Model's predicted next token: ' fence'


In [81]:
from inseq import load_model
import torch
import torch.nn.functional as F
# 1. Load model with gradient-based attribution
model = load_model("gpt2", "saliency", device="cuda" if torch.cuda.is_available() else "cpu")
embedding_layer = model.model.get_input_embeddings()

# 2. Define input and target output token
input_text = "The quick brown fox jumps over the"
target_token = " barking"  # Token we want to make more favorable

# 3. Get token IDs
inputs = model.tokenizer(input_text, return_tensors="pt")
target_id = model.tokenizer.convert_tokens_to_ids(target_token)

# # For optimizing toward a phrase:
# target_ids = [model.tokenizer.convert_tokens_to_ids(t) for t in ["lazy", "dog"]]
# target_logits = sum(outputs.logits[0, -len(target_ids):, target_ids].diag())

if target_id == model.tokenizer.unk_token_id:
     print(f"Warning: Target token '{target_token}' not found in vocabulary. Using UNK ID ({target_id}). Results might not be meaningful.")

# 4. Perform forward pass with gradient tracking
with torch.set_grad_enabled(True):
    # Get embeddings and mark as requiring gradients
    embeddings = embedding_layer(inputs.input_ids)
    embeddings.retain_grad()
    embeddings.requires_grad_(True)
    
    # Forward pass through model
    outputs = model.model(
        inputs_embeds=embeddings,
        attention_mask=inputs.attention_mask
    )
    
    # Get logits for the target position (next token prediction)
    target_position = -1  # Position after last input token
    target_logit = outputs.logits[0, target_position, target_id]
    
    # Compute gradients
    target_logit.backward()
    gradients = embeddings.grad.clone()

# # 5. Modify embeddings with gradients
modified_embeddings = embeddings + gradients

all_embeddings = embedding_layer.weight #Vocab Embeddings
new_token_ids = []
for i, emb in enumerate(modified_embeddings[0]) : 
    distances = F.cosine_similarity(emb, all_embeddings, dim=-1)
    new_id = torch.argmax(distances).item()
    new_token_ids.append(new_id)

#Decode the new tokens
new_tokens = model.tokenizer.convert_ids_to_tokens(new_token_ids)
new_text = model.tokenizer.convert_tokens_to_string(new_tokens)
print(f"Modified text: {new_text}")

# 8 Compare Probabilities
with torch.no_grad():
    modified_outputs = model.model(
        inputs_embeds=modified_embeddings,
        attention_mask=inputs.attention_mask
    )
    original_probs = torch.softmax(outputs.logits[0, target_position], dim=-1)
    modified_probs = torch.softmax(modified_outputs.logits[0, target_position], dim=-1)

print(f"Original probability of '{target_token}': {original_probs[target_id].item():.10f}")
print(f"Modified probability of '{target_token}': {modified_probs[target_id].item():.10f}")


# Show token substitutions
print("\nToken changes: ")
for orig, new in zip(model.tokenizer.tokenize(input_text), new_tokens):
    if orig != new: 
        print(f"{orig} -> {new}")
    else : 
        print("No change :", orig)
# # Optional: Visualize gradient magnitudes
# gradient_magnitudes = torch.norm(gradients, dim=-1).squeeze()
# print("\nGradient magnitudes per input token:")
# for token, magnitude in zip(model.tokenizer.convert_ids_to_tokens(inputs.input_ids[0]), gradient_magnitudes):
#     print(f"{token:>10}: {magnitude.item():.4f}")

Modified text: The quick brown fox jumps over the
Original probability of ' barking': 0.0000003711
Modified probability of ' barking': 0.0004909183

Token changes: 
No change : The
No change : Ġquick
No change : Ġbrown
No change : Ġfox
No change : Ġjumps
No change : Ġover
No change : Ġthe


In [58]:
len(embeddings[0])

8

In [55]:
len(gradients[0])

8

In [1]:
import torch
# import inseq
from inseq import load_model
# from inseq.models import InseqModel
from transformers import AutoTokenizer, AutoModelForCausalLM

c:\Users\Kalai\anaconda3\envs\XAI\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
MODEL_NAME = 'gpt2'
INPUT_TEXT = "The quick fox jumps over the lazy "
TARGET_OUTPUT_TOKEN = "dog"
TARGET_OUTPUT_POSITION = -1

In [21]:
model = load_model("gpt2", "saliency", device="cuda" if torch.cuda.is_available() else "cpu")

c:\Users\Kalai\anaconda3\envs\XAI\Lib\site-packages\transformers\generation\configuration_utils.py:820: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_attentions` is. When `return_dict_in_generate` is not `True`, `output_attentions` is ignored.
  warnings.warn(


In [22]:
input_token = model.tokenizer(INPUT_TEXT, return_tensors='pt')
input_ids = input_token["input_ids"].to(model.device)
attention_mask = input_token["attention_mask"].to(model.device)

In [24]:
target_token_id = model.tokenizer.convert_tokens_to_ids(TARGET_OUTPUT_TOKEN)
if target_token_id == model.tokenizer.unk_token_id: 
    print(f"Warning: Target token '{TARGET_OUTPUT_TOKEN}' not found in vocabulary. Using UNK ID ({target_token_id}). Results might not be meaningful.")

print(f"Target token '{TARGET_OUTPUT_TOKEN}' corresponds to ID: {target_token_id}")

Target token 'dog' corresponds to ID: 9703


In [47]:
print(model.model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


In [45]:
# get input embeddings
# with torch.set_grad_enabled(True) : 
original_embeddings = model.model.get_input_embeddings()(input_ids)
original_embeddings.clone().detach().requires_grad_(True)

if hasattr(model.model, 'encoder') and hasattr(model.model, 'decoder'): 
    print("Detected Seq2Seq model")